In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Mesothelioma/GSE117668'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from in vitro healthy cells and malignant pleural mesothelioma cell lines infected by oncolytic attenuated measles virus or treated by exogenous type I interferon"
!Series_summary	"We used microarrays to analyse antiviral response by mesothelioma cells sensitive or resistant to the oncolytic activity of measles virus"
!Series_overall_design	"4 healthy cells and 12 malignant pleural mesothelioma cell lines were infected with measles virus (MV, MOI 1) or not infected (NI = basal expression) or were treated with type I interferon (IFN : IFN-alpha2 + IFN-Beta, 1000UI/mL) during 48 hours. Cells were lysed and RNA was extracted of each sample. This experiment was performed 3 times. RNA was quantified after purification and same quantity of RNA of the 3 experiments was pooled in the same tube for each sample. Then, RNA was analyzed with microarrays."
Sample Characteristics Dictionary:
{0: ['cell type: lung epithelial cells', 'cell type: f

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True

# Identify the keys for 'Mesothelioma', 'age', and 'gender'
sample_characteristics = {
    0: ['cell type: lung epithelial cells', 'cell type: fibroblasts', 'cell type: endothelial cells', 'cell type: peritoneal mesothelial cells', 'cell type: MPM cell line'],
    1: ['diagnosis: healthy', 'diagnosis: malignant pleural mesothelioma\xa0']
}

# Determine rows for each variable
trait_row = 1 if 'diagnosis: malignant pleural mesothelioma\xa0' in sample_characteristics[1] else None
age_row = None  # No indication of age data in the sample characteristics dictionary
gender_row = None  # No indication of gender data in the sample characteristics dictionary

# Define the conversion functions
def convert_trait(value):
    if "diagnosis: malignant pleural mesothelioma" in value:
        return 1
    elif "diagnosis: healthy" in value:
        return 0
    else:
        return None

# No conversion functions needed for age and gender as the data is not available
def convert_age(value):
    return None

def convert_gender(value):
    return None

# Save cohort info
save_cohort_info('GSE117668', './preprocessed/Mesothelioma/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical feature data if trait data is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Mesothelioma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Mesothelioma/trait_data/GSE117668.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3305861': [0], 'GSM3305862': [0], 'GSM3305863': [0], 'GSM3305864': [0], 'GSM3305865': [0], 'GSM3305866': [0], 'GSM3305867': [0], 'GSM3305868': [0], 'GSM3305869': [0], 'GSM3305870': [0], 'GSM3305871': [0], 'GSM3305872': [0], 'GSM3305873': [1], 'GSM3305874': [1], 'GSM3305875': [1], 'GSM3305876': [1], 'GSM3305877': [1], 'GSM3305878': [1], 'GSM3305879': [1], 'GSM3305880': [1], 'GSM3305881': [1], 'GSM3305882': [1], 'GSM3305883': [1], 'GSM3305884': [1], 'GSM3305885': [1], 'GSM3305886': [1], 'GSM3305887': [1], 'GSM3305888': [1], 'GSM3305889': [1], 'GSM3305890': [1], 'GSM3305891': [1], 'GSM3305892': [1], 'GSM3305893': [1], 'GSM3305894': [1], 'GSM3305895': [1], 'GSM3305896': [1], 'GSM3305897': [1], 'GSM3305898': [1], 'GSM3305899': [1], 'GSM3305900': [1], 'GSM3305901': [1], 'GSM3305902': [1], 'GSM3305903': [1], 'GSM3305904': [1], 'GSM3305905': [1], 'GSM3305906': [1], 'GSM3305907': [1], 'GSM3305908': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033413_at', '100033422_at', '100033423_at', '100033424_at',
       '100033425_at', '100033426_at', '100033427_at', '100033428_at',
       '100033430_at', '100033431_at', '100033432_at', '100033434_at',
       '100033435_at', '100033436_at', '100033437_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at'], 'SPOT_ID': ['100009613', '100009676', '10000', '10001', '10002'], 'Description': ['ANO1 antisense RNA 2 (head to head)', 'ZBTB11 antisense RNA 1', 'AKT serine/threonine kinase 3', 'mediator complex subunit 6', 'nuclear receptor subfamily 2 group E member 3']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate keys in the gene annotation dictionary
identifier_key = 'ID'
gene_symbol_key = 'Description'

# 2. Get the mapping dataframe using the 'get_gene_mapping' function from the library
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Mesothelioma/gene_data/GSE117668.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Mesothelioma')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE117668', './preprocessed/Mesothelioma/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Mesothelioma/GSE117668.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Mesothelioma', the least common label is '0.0' with 12 occurrences. This represents 25.00% of the dataset.
The distribution of the feature 'Mesothelioma' in this dataset is fine.

